In [1]:
import scipy
import scipy.sparse
import cPickle
import logging
import implicit
reload(implicit)
import implicit.approximate_als
reload(implicit.approximate_als)
import implicit.bpr
reload(implicit.bpr)
import sklearn
import numpy as np
import itertools
import random
import pandas as pd
import time
import gc
import lightfm
import lightfm.evaluation
import tqdm
import seaborn as sns
%matplotlib inline
import util as C
reload(C)
import matplotlib.pyplot as plt

# Load Data

In [2]:
logging.basicConfig(level = logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
user_helper = C.get_user_helper()

In [4]:
df_idx = user_helper.load_evst_subs_idx_df()

In [5]:
df_idx.head(3)

,uid,cs
0,d4ef183e7b34430f91707dfec7e12dd1,11414
1,6c6e4d6f97904e018b06643032d4e85d,399+2465+3308+5950+13798+49689+69215+71013+105...
2,726554550003492895e92395a8ef8943,3421+11413+11554+26353+38381+40442+52917+67079...


In [6]:
%%time
mat_csr = C.load_mat(C.EVST_USER_ITEM_PATH)

CPU times: user 164 ms, sys: 96 ms, total: 260 ms
Wall time: 256 ms


In [7]:
mat_csr

<1884585x137344 sparse matrix of type '<type 'numpy.float32'>'
	with 15979221 stored elements in Compressed Sparse Row format>

# Split Train/Test Data

In [8]:
MIN_SUBS = 6
MAX_SUBS = 10
TRAIN_USERS = 20 * 10000
train_mat_csr, train_users_idx = C.build_train_matrix(mat_csr, n_users = TRAIN_USERS, min_subs=MIN_SUBS, max_subs=MAX_SUBS)

  0%|          | 851/200000 [00:00<00:23, 8500.83it/s]

# of all users = 1884585
# of ok users = 372377
# of train users = 200000


100%|██████████| 200000/200000 [00:17<00:00, 11383.55it/s]


In [9]:
%%time
train_mat_coo = train_mat_csr.tocoo()

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.3 ms


In [10]:
train_meta_df = df_idx.iloc[train_users_idx].copy()
train_meta_df['subs'] = train_meta_df['cs'].astype(str).apply(lambda x: len(x.split('+')))
train_meta_df.reset_index(inplace=True)
train_meta_df.drop(['cs'], axis=1, inplace=True)
train_meta_df.rename_axis({'index':'origin_idx'}, axis = 1, inplace=True)

In [11]:
train_meta_df.head(3)

,origin_idx,uid,subs
0,1749024,f1a00626773a4a5bbca733b78c1dacd8,6
1,654302,b21b7abd64104875924f24ededacb637,6
2,779381,df9e4eaf667749d38de89efe1f33216d,8


In [12]:
train_meta_df.to_csv(C.EVST_TRAIN_USER_ITEM_META_PATH)

In [13]:
!head data/evst_train_user_item_meta.csv

,origin_idx,uid,subs
0,1749024,f1a00626773a4a5bbca733b78c1dacd8,6
1,654302,b21b7abd64104875924f24ededacb637,6
2,779381,df9e4eaf667749d38de89efe1f33216d,8
3,581919,094e8f67ecd84d05bd5c4d233cde44d7,9
4,1417550,7623967ffe1e43f1aaa6f785ac57b4c6,10
5,575623,46a475dfecfd47f6b130f171ec7a2f05,6
6,159417,52c4a461d2454f2e8160201ce25aa256,7
7,1429320,29dbb84b9e6a43f3b3817227119b1584,10
8,803571,ee09aa4c54594cdca723fc159b1190c8,8


In [14]:
TEST_USERS = 5 * 10000
train_mat_csr2, test_mat_csr, test_users_idx = C.train_test_split(train_mat_csr, n_test_users=TEST_USERS)

  1%|▏         | 721/50000 [00:00<00:06, 7207.12it/s]

# of test users = 50000


100%|██████████| 50000/50000 [00:06<00:00, 7417.63it/s]


In [15]:
test_meta_df = train_meta_df.iloc[test_users_idx]
test_meta_df.to_csv(C.EVST_TEST_USER_ITEM_META_PATH)

In [16]:
test_meta_df.head(3)

,origin_idx,uid,subs
119737,1738927,8d0ad848e58d47718bc9048fe55e9941,6
72272,1835513,fb3c8041026043918bacedda05361106,7
158154,1790575,be64359ae3204dc1a550bf285f3896b0,10


In [17]:
!head data/evst_test_user_item_meta.csv

,origin_idx,uid,subs
119737,1738927,8d0ad848e58d47718bc9048fe55e9941,6
72272,1835513,fb3c8041026043918bacedda05361106,7
158154,1790575,be64359ae3204dc1a550bf285f3896b0,10
65426,50023,3bbf9743aa12449b9d73cbbbad19d073,6
30074,809872,121461a8a74c49f1b34519d1b6430c7f,9
23677,394925,94c41c32b0df4b568154a703017c537d,7
134858,1806054,91e33c082c4b4ca082ca69e95280ab55,6
176418,1741419,7d612c1e1c5f43e4a1e3e158bc4b4543,9
132467,1243487,de33d71a4a2a406da5f0d98046431822,9


# Save Data

In [18]:
C.save_mat(C.EVST_TRAIN_USER_ITEM_MAT_PATH, train_mat_csr2, compressed=False)
C.save_mat(C.EVST_TEST_USER_ITEM_MAT_PATH, test_mat_csr, compressed=False)